In [9]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI

vector_store = Chroma(
    collection_name="my_database",
    embedding_function=GoogleGenerativeAIEmbeddings(model="models/embedding-001"),  
    persist_directory="chroma",
)

In [12]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5},
)

In [13]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")

In [14]:
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

prompt = PromptTemplate.from_template(
    "你是 Rasmus 的履歷助手，負責依據資料庫回覆使用者的問題。如果你不知道，請直接說不知道。\n **context**: {context}\n\n **input**: {input}\n\n"
)

combine_docs_chain = create_stuff_documents_chain(
    llm, prompt
)

retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [15]:
retrieval_chain.invoke(
    {"input": "Rasmus 如何開啟對行銷的興趣？"}
)

{'input': 'Rasmus 如何開啟對行銷的興趣？',
 'context': [Document(id='info_self_introduction.md_1', metadata={'chunk_index': 1, 'source': 'info_self_introduction.md'}, page_content='---h2--- 我的故事：在理性與感性之間成長\n我叫朱畇熙，英文名 Rasmus，這個名字背後有個有趣的故事。我的父親是位嚴謹的會計師，從小教育我「數字不會說謊」；母親則是位熱愛文學與藝術的老師，鼓勵我「故事能感動人心」。\n我成長在這樣一個理性與感性交織的家庭，學會了用嚴謹的眼光觀察世界，也學會了用溫暖的筆觸描繪生活。或許正是因為如此，我在面對會計系密密麻麻的數字時，總能想起母親教我的故事結構；而在撰寫行銷文案時，父親的數字思維又能幫助我評估成效。'),
  Document(id='info_self_introduction.md_0', metadata={'chunk_index': 0, 'source': 'info_self_introduction.md'}, page_content='---h1--- 關於 Rasmus Zhu：從會計數字到品牌敘事'),
  Document(id='project_marketing_comp.pdf_0', metadata={'chunk_index': 0, 'source': 'project_marketing_comp.pdf'}, page_content='專案主題：IKEA 學生市場推廣行銷企劃\n\n標題：任務\n內容：在行銷管理課程中，我們被賦予任務，為 IKEA 規劃一套針對中部學生市場的行銷方案，以推廣其學生家具系列。'),
  Document(id='project_marketing_proposal.pdf_1', metadata={'chunk_index': 1, 'source': 'project_marketing_proposal.pdf'}, page_content='**標題：情境**\n**內容：** 參與全國大專行銷挑戰賽，為「XYZ 能量飲料」在競爭激烈的市場中，提出創新的行銷策略，吸引年輕客群。\n\n**標